In [18]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import csv
import time
from os.path import isfile
import pandas as pd
import re
import spacy
import matplotlib.pyplot as plt
import Tools.processing as proc

model = SentenceTransformer('all-mpnet-base-v2')

In [19]:
titles_df = ["Text", "Score"]
df_amazon = pd.read_csv("/home/marcuswrrn/Projects/Semantic_Quantification/Semantic_Comparison/Lab_Tests/Data_Storage/Positive_Scores/sentiment labelled sentences/amazon_cells_labelled.txt", names=titles_df, sep='\t')
df_imbd = pd.read_csv("/home/marcuswrrn/Projects/Semantic_Quantification/Semantic_Comparison/Lab_Tests/Data_Storage/Positive_Scores/sentiment labelled sentences/imdb_labelled.txt", names=titles_df, sep='\t')
df_yelp = pd.read_csv("/home/marcuswrrn/Projects/Semantic_Quantification/Semantic_Comparison/Lab_Tests/Data_Storage/Positive_Scores/sentiment labelled sentences/yelp_labelled.txt", names=titles_df, sep='\t')

In [20]:
encodings = df_amazon["Text"].apply(lambda x: model.encode(x))
encodings2 = df_imbd["Text"].apply(lambda x: model.encode(x))
encodings3 = df_yelp["Text"].apply(lambda x: model.encode(x))

In [21]:
few_shot = pd.read_json("/home/marcuswrrn/Projects/Semantic_Quantification/Semantic_Comparison/Data_Processing/Data/job_pooled_embeddings.json")
positive = [float(text) for text in few_shot["positive"]]
negative = [float(text) for text in few_shot["negative"]]

In [22]:
def compare_embeddings(embed, positive=few_shot['positive'], negative=few_shot['negative'], hyper=0.2, visible=False):
    embed_temp = np.array([float(x) for x in embed], dtype=np.float32)
    positive = np.array([float(x) for x in positive], dtype=np.float32)
    negative = np.array([float(x) for x in negative], dtype=np.float32)

    similarity_pos = util.cos_sim(embed_temp, positive)[0][0]
    similarity_neg = util.cos_sim(embed_temp, negative)[0][0]
    #print(f"{similarity_pos} : {similarity_neg} : {similarity_pos >= similarity_neg}")
    diff = similarity_pos - (similarity_neg + (similarity_pos *hyper))
    if visible:
        print(f"Difference in embedding {diff}")
    return (diff >= 0).item()
#df['Encodings'] = [float(x) for x in df["Encodings"]]
#df["Test_Score"] = df['Encodings'].apply(lambda x: compare_embeddings(x))

#encodings = encodings.astype(np.float32)
df_amazon["Train_Score"] = encodings.apply(lambda x: compare_embeddings(x, hyper=0))
df_imbd["Train_Score"] = encodings2.apply(lambda x: compare_embeddings(x, hyper=0))
df_yelp["Train_Score"] = encodings3.apply(lambda x: compare_embeddings(x, hyper=0))

In [23]:
train_scores = df_amazon["Train_Score"]
test_scores = df_amazon["Score"]

def build_confusion_matrix(train_scores, test_scores):
    matrix = np.zeros((2, 2))
    for train, test in zip(train_scores, test_scores):
        train_temp = 1 if train else 0
        test_temp = 1 if test else 0
        #print(f"Train: {train}, Test: {test}")
        matrix[test_temp][train_temp] += 1
    return matrix

def find_test_score(train_score, test_score):
    count = 0
    for test, train in zip(test_score, train_score):
        count += 1 if test == train else 0
    return count/len(test_score)

def binary_confusion_matrix_scores(confusion_matrix):
    """
    Compute precision, accuracy, and recall scores from a 2x2 confusion matrix.
    
    Args:
        confusion_matrix (list of lists): A 2x2 list containing the confusion matrix, 
                                          with rows representing true labels and columns representing predicted labels.
    
    Returns:
        precision (float): Precision score for the positive class.
        accuracy (float): The overall accuracy of the classifier.
        recall (float): Recall score for the positive class.
    """
    
    # Make sure the input is a 2x2 matrix
    if len(confusion_matrix) != 2 or any(len(row) != 2 for row in confusion_matrix):
        raise ValueError("The confusion matrix must be 2x2")
    
    # Calculate true positives, false positives, and false negatives
    true_positives = confusion_matrix[1][1]
    false_positives = confusion_matrix[0][1]
    false_negatives = confusion_matrix[1][0]
    true_negatives = confusion_matrix[0][0]
    
    # Compute precision, accuracy, and recall scores
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) != 0 else 0
    accuracy = (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_negatives)
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) != 0 else 0
    print(f"Precision: {precision}")
    print(f"Accuracy: {accuracy}")
    print(f"Recall: {recall}")

def print_test_log(df : pd.DataFrame, train_label="Train_Score", test_label="Score"):
    
    conf_matrix = build_confusion_matrix(df[train_label], df[test_label])
    print(conf_matrix)
    binary_confusion_matrix_scores(conf_matrix)

def print_all_logs(df_amazon, df_imbd, df_yelp):
    print("Amazon")
    print_test_log(df_amazon)
    print()
    print("IMBD")
    print_test_log(df_imbd)
    print()
    print("Yelp")
    print_test_log(df_yelp)

print_all_logs(df_amazon, df_imbd, df_yelp)

Amazon
[[397. 103.]
 [ 60. 440.]]
Precision: 0.8103130755064457
Accuracy: 0.837
Recall: 0.88

IMBD
[[328.  34.]
 [ 44. 342.]]
Precision: 0.9095744680851063
Accuracy: 0.8957219251336899
Recall: 0.8860103626943006

Yelp
[[405.  95.]
 [ 22. 478.]]
Precision: 0.8342059336823735
Accuracy: 0.883
Recall: 0.956


In [24]:
# Compare to single sentence embedding
sents_pos = ["This is a positive sentence.", "I am so Happy!!!", "The world is good!"]
encs_pos = model.encode(sents_pos)

sents_neg = ["This is a negative sentence.", "I am so Sad.", "The world is a terrible place."]
encs_neg = model.encode(sents_neg)

for i in range(len(sents_pos)):
    print(f"+Sentence {i}+")
    df_amazon["Train_Score"] = encodings.apply(lambda x: compare_embeddings(x, positive=encs_pos[i], negative=encs_neg[i], hyper=0))
    df_imbd["Train_Score"] = encodings2.apply(lambda x: compare_embeddings(x, positive=encs_pos[i], negative=encs_neg[i], hyper=0))
    df_yelp["Train_Score"] = encodings3.apply(lambda x: compare_embeddings(x, positive=encs_pos[i], negative=encs_neg[i], hyper=0))
    print_all_logs(df_amazon, df_imbd, df_yelp)
    print()
    



+Sentence 0+
Amazon
[[278. 222.]
 [ 38. 462.]]
Precision: 0.6754385964912281
Accuracy: 0.74
Recall: 0.924

IMBD
[[262. 100.]
 [ 24. 362.]]
Precision: 0.7835497835497836
Accuracy: 0.8342245989304813
Recall: 0.9378238341968912

Yelp
[[294. 206.]
 [ 15. 485.]]
Precision: 0.7018813314037626
Accuracy: 0.779
Recall: 0.97

+Sentence 1+
Amazon
[[430.  70.]
 [115. 385.]]
Precision: 0.8461538461538461
Accuracy: 0.815
Recall: 0.77

IMBD
[[341.  21.]
 [116. 270.]]
Precision: 0.9278350515463918
Accuracy: 0.8168449197860963
Recall: 0.6994818652849741

Yelp
[[452.  48.]
 [ 83. 417.]]
Precision: 0.896774193548387
Accuracy: 0.869
Recall: 0.834

+Sentence 2+
Amazon
[[364. 136.]
 [ 52. 448.]]
Precision: 0.7671232876712328
Accuracy: 0.812
Recall: 0.896

IMBD
[[327.  35.]
 [ 59. 327.]]
Precision: 0.9033149171270718
Accuracy: 0.8743315508021391
Recall: 0.8471502590673575

Yelp
[[377. 123.]
 [ 17. 483.]]
Precision: 0.7970297029702971
Accuracy: 0.86
Recall: 0.966



In [25]:
# df_standford = pd.read_csv("/home/marcuswrrn/Projects/Semantic_Quantification/Semantic_Comparison/Lab_Tests/Data_Storage/stanfordSentimentTreebank/stanfordSentimentTreebank/datasetSentences.txt", sep='\t')
# df_standford['Score'] = pd.read_csv("/home/marcuswrrn/Projects/Semantic_Quantification/Semantic_Comparison/Lab_Tests/Data_Storage/stanfordSentimentTreebank/stanfordSentimentTreebank/datasetSplit.txt")['splitset_label']

# df_standford["Encodings"] = df_standford["sentence"].apply(lambda x: model.encode(x))
# df_standford.head()

In [26]:
# df_standford["Score"] = pd.read_csv("/home/marcuswrrn/Projects/Semantic_Quantification/Semantic_Comparison/Lab_Tests/Data_Storage/stanfordSentimentTreebank/stanfordSentimentTreebank/sentiment_labels.txt", sep="|")['sentiment values']
# df_standford.to_json("./Data_Storage/Processed_Data/standford_sentiment.json")

In [27]:
df_standford = pd.read_json("./Data_Storage/Processed_Data/standford_sentiment.json")
df_standford["Label"] = df_standford["Score"]
df_standford["Score"] = df_standford["Score"].apply(lambda x: x > 0.5)
print(df_standford.shape)
print(df_standford["sentence"][48])
df_standford.head(50)

(11855, 6)
Gosling provides an amazing performance that dwarfs everything else in the film .


,sentence_index,sentence,Score,Encodings,Train_Score,Label
0,1,The Rock is destined to be the 21st Century 's...,False,"[0.0105858203, -0.0443099923, 0.0058608577, -0...",True,0.50000
1,2,The gorgeously elaborate continuation of `` Th...,False,"[0.040952641500000005, 0.0028768007, -0.027633...",True,0.50000
2,3,Effective but too-tepid biopic,False,"[0.0409016721, 0.110521771, 0.0246010255000000...",True,0.44444
3,4,If you sometimes like to go to the movies to h...,False,"[-0.0031407475000000003, 0.0303974543, -0.0181...",False,0.50000
4,5,"Emerges as something rare , an issue movie tha...",False,"[0.0090491241, 0.0368086249, 0.0298471693, 0.0...",True,0.42708
5,6,The film provides some great insight into the ...,False,"[0.0061802664, 0.059488479000000004, 0.0159128...",True,0.37500
6,7,Offers that rare combination of entertainment ...,False,"[-0.015778821000000002, 0.059646632500000005, ...",True,0.41667
7,8,Perhaps no picture ever made has more literall...,True,"[-0.029814334600000002, 0.102527298, -0.022574...",True,0.54167
8,9,Steers turns in a snappy screenplay that curls...,False,"[-0.0278737657, -0.0428418629, 0.0119619695, 0...",True,0.33333
9,10,But he somehow pulls it off .,False,"[0.0287389942, 0.054299406700000004, -0.013576...",False,0.45833


In [28]:
def binary_confusion_matrix_scores(confusion_matrix):
    """
    Compute precision, accuracy, and recall scores from a 2x2 confusion matrix.
    
    Args:
        confusion_matrix (list of lists): A 2x2 list containing the confusion matrix, 
                                          with rows representing true labels and columns representing predicted labels.
    
    Returns:
        precision (float): Precision score for the positive class.
        accuracy (float): The overall accuracy of the classifier.
        recall (float): Recall score for the positive class.
    """
    
    # Make sure the input is a 2x2 matrix
    if len(confusion_matrix) != 2 or any(len(row) != 2 for row in confusion_matrix):
        raise ValueError("The confusion matrix must be 2x2")
    
    # Calculate true positives, false positives, and false negatives
    true_positives = confusion_matrix[1][1]
    false_positives = confusion_matrix[0][1]
    false_negatives = confusion_matrix[1][0]
    true_negatives = confusion_matrix[0][0]
    
    # Compute precision, accuracy, and recall scores
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) != 0 else 0
    accuracy = (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_negatives)
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) != 0 else 0
    print(f"Precision: {precision}")
    print(f"Accuracy: {accuracy}")
    print(f"Recall: {recall}")



df_standford["Train_Score"] = df_standford["Encodings"].apply(lambda x: compare_embeddings(x, hyper=0))
conf_matrix = build_confusion_matrix(df_standford["Train_Score"], df_standford["Score"])
print(conf_matrix)
binary_confusion_matrix_scores(conf_matrix)
hyper_param = 0.2
print()
df_standford["Train_Score"] = df_standford["Encodings"].apply(lambda x: compare_embeddings(x, positive=encs_pos[1], negative=encs_neg[1], hyper=0))
conf_matrix = build_confusion_matrix(df_standford["Train_Score"], df_standford["Score"])
print(conf_matrix)
binary_confusion_matrix_scores(conf_matrix)





[[2742. 4000.]
 [1997. 3116.]]
Precision: 0.4378864530635188
Accuracy: 0.49413749472796287
Recall: 0.6094269509094465

[[4278. 2464.]
 [3146. 1967.]]
Precision: 0.4439178515007899
Accuracy: 0.5267819485449178
Recall: 0.38470565225894776
